## Load book
### Note this has to be ran per book

In [ ]:
books = ['Outlander', 'Dragonfly in Amber', 'Voyager', 'Drums of Autumn', 'The Fiery Cross', 'A Breath of Snow and Ashes',
             'An Echo in the Bone', 'Written in My Own Heart’s Blood']
extras = ['Other Books by this Author', 'About the Author']
bookstarts = [50, 17287, 37378, 61857, 89432, 119494, 152540, 177800, 202059]

In [ ]:
f = open("data.txt", "r", encoding="utf8")
booknum = 1


book = ''
i = 0
for line in f:
    if i < bookstarts[booknum] and i > bookstarts[booknum-1]:
        book = book + line
    i = i + 1
print(len(book))

In [ ]:
from flair.data import Sentence, Token
from flair.models import SequenceTagger

# load tagger
tagger_ner = SequenceTagger.load("flair/ner-english-fast")
tagger_pos = SequenceTagger.load("flair/pos-english")

## Tagging and splitting code

In [ ]:
def ner_tag(text):
    sentence_ner = Sentence(text)

    # predict NER tags
    tagger_ner.predict(sentence_ner)
    return sentence_ner

In [ ]:
def pos_tag(text):
   sentence_pos = Sentence(text.split(' '), use_tokenizer=True)

   # predict NER tags
   tagger_pos.predict(sentence_pos)
   return sentence_pos

In [ ]:
def get_person_before(sentence_ner, token):
    # If text is 'it' then don't change it
    if token.text=='it':
        return token.text
    # If text is 'I' or 'me' then input Claire as it's first person
    if token.text=='I' or token.text=='me':
        return 'Claire'
    prev_person = ''
    # Else input the previous name
    for entity in sentence_ner.get_spans('ner'):
        if entity.tag == 'PERSON' and entity.start_position<token.start_position:
                prev_person = entity.text
        elif entity.start_position>token.start_position:
            break
    return prev_person

In [ ]:
import re
def split_into_chapters(booknum, text):
    if booknum <7:
        return re.split(r'(\\n[0-9]+\\n\\n)', text)
    else:
        # needs refinement
        return re.split(r"\n(?=[A-Z ]+\n)", text)

In [ ]:
def split_into_paragraphs(text):
    splitted = re.split('(\*\*\*)', text)
    return [x for x in splitted if x != '']

## Replace and write to file

In [ ]:
chapters = re.split(r'\n[0-9]+\n\n', book)
i = 0
for chapter in chapters:
    paragraphs = split_into_paragraphs(chapter)
    for paragraph in paragraphs:
            print('Chapter ', i)
            print('Pos tagging....')
            sentence_pos = pos_tag(paragraph)
            print('Ner tagging....')
            sentence_ner = ner_tag(paragraph)
            no_pronouns_book = Sentence(paragraph.split(' '), use_tokenizer=True)
            print('Replacing pronouns....')
            i=0
            for token in sentence_pos:
                if token.tag == 'PRP':
                    name = get_person_before(sentence_ner, token)
                    new_token = Token(name)
                    no_pronouns_book.tokens[token.idx-1] = new_token
            with open('data_no_pronouns.txt', 'a') as f:
                f.write(no_pronouns_book.to_original_text())
    i+=1

In [ ]:
c = re.split(r'\n[0-9]+\n\n', book)
c[0]